[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/rodreras/geopy_minicurso/HEAD)

# Primeiros Passos: Remote Sensing + Python

Sensoriamento Remoto é muito útil em trabalhos de geologia, e outra ciências ambientais.

Porém, nem sempre é tão trivial lidar com dados raster em Python, já que você começa a lidar com um maior número de dimensões, dificultando a compreensão. 

Aqui, vamos dar uma pincelada para que você tenha uma ideia inicial e depois possa ir buscar mais conteúdo sobre isso. Saiba que existe um universo de dados e conteúdo para explorar e aplicar nos seus estudos. 



In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#biblioteca para lidar com dados raster
import rasterio as rio
from rasterio.plot import show, show_hist, reshape_as_raster

import earthpy.spatial as es
import earthpy.plot as ep
import earthpy as et

#para baixar a imagem
import urllib.request
import os

In [ ]:
#como a imagem é grande, e não é recomendável deixar no repositório
#eu salvei no meu gdrive, e através da biblioteca request
#estamos fazendo o donwload dela para a pasta Dados.

url_imagem = 'http://rodrigobrust.com/wp-content/uploads/2023/06/landsat5_empilhada.tif'

os.mkdir('./Dados/')

caminho = "./Dados/"  

nome_arquivo = 'landsat5_empilhada.tif'  # definindo o nome do arquivo

salvando = os.path.join(caminho, nome_arquivo)  # criando o local para salvar

urllib.request.urlretrieve(url_imagem, salvando)  # Fazendo Download e salvando

print("Arquivo baixado com sucesso!")

In [ ]:
#aqui já tem um arquivo raster com 5 bandas empilhadas.
img = './Dados/landsat5_empilhada.tif'

In [ ]:
#abrindo raster empilhado
with rio.open(img) as src:
    bandas = src.read()
    #copiando os metadados da imagem raster
    out_meta = src.profile



In [ ]:
#usando a funçao do earthplot para plotar todas as bandas do arquivo
ep.plot_bands(bandas,
              cbar=False,
              cmap = 'gist_earth')
plt.show()

## Fazendo as primeiras composições

Uma coisa importante! Por mais que as bandas sejam de 1 a 6, quando fizermos o código para plotar, temos que pensar que o Python trabalha com índices. Ou seja, as bandas serão entendidas de 0 a 5.

In [ ]:
#criando um grid de 2 figuras
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(10,8), sharey=True)

#figura 0,0
ep.plot_rgb(bandas,
            rgb=[2,3,1],
            stretch=True,
            figsize=(5, 6),
            ax=ax[0],
            title="RGB Para Agricultura")

#figura 0,1
ep.plot_rgb(bandas,
            rgb=[2,1,0],
            stretch=True,
            figsize=(5, 6),
            ax=ax[1],
            title="RGB True Color Image")

plt.show()

# Gerando Histogramas para cada banda

In [ ]:
ep.hist(bandas,
        title=[f'Banda-{i+1}' for i in range(1, 7)], 
        cols=3, 
        alpha=0.5, 
        figsize = (12, 10))

plt.show()

# Salvando imagem

In [ ]:
#empilhando as bandas com numpy
rgb = np.stack((bandas[2], bandas[1], bandas[0]))

In [ ]:
# atualizando metadata
out_meta.update({"driver": "GTiff",
            "height": bandas.shape[1],
            "width": bandas.shape[2],
            "count":3,
            })



# salvando o arquvi
with rio.open('./Dados/RGB_image.tif', "w", **out_meta) as dest:
   
    dest.write(rgb, #aqui, colocamos onde a imagem foi empilhada, no caso rgb
                [1,2,3] #aqui indicamos a quantidade de bandas e a ordem
                )

## Teste! 

Agora que você já exportou a sua imagem, abra o QGIS e carregue a imagem. Ela vai abrir já como RGB composição cor verdadeira.